The CE888 Assignment 1:

A combination of Neural Networks (NNs) and Monte Carlo Tree Search (MCTS) has resulted in really strong players for a multitude of games. The basic idea is to create a learning loop with the following steps:
1. Multiple games are played using a standard search algorithm like MCTS.
2. The actions that the player took at each state are kept and used to train a neural network. 
3. The neural network is used to enhance/guide the search algorithm.

For this project we are going to implement a simpler version of this algorithm (no neural network included!) using decision trees as the classiﬁer.





The original code : https://drive.google.com/file/d/1aeBVSFMlwVUqqnqta3wU-5tG5iiVkpI8/view


In [4]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Example GameState classes for Nim, OXO and Othello are included to give some idea of how you
# can write your own GameState use UCT in your 2-player game. Change the game to be played in 
# the UCTPlayGame() function at the bottom of the code.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

from math import *
import random

class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement UCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass


class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        #return "["+str(self.move)+ str(self.wins)+ str(self.visits)+ str(self.untriedMoves)+"]"
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""


    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    if verbose:
        #print(rootnode.TreeToString(0))
        v = rootnode.TreeToString(0)
    else: 
        #print(rootnode.ChildrenToString())
        v = rootnode.ChildrenToString()

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited
    # Can add ' v


In [5]:
class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move] = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm:
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []: return 0.5 # draw
        return False # Should not be possible to get here

    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        return s



In [8]:
#The first part which worked


def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    win = 0
    best = []
    currentplay = []
    brd =[]
    l=[]
    state = OXOState() # uncomment to play OXO
    while state.GetMoves() != []:
        print(str(state))
        st = state
        p = 3 - state.playerJustMoved
        if state.playerJustMoved == 1:
            m = UCT(rootstate=state, itermax=1000, verbose=False)  # play with values for itermax and verbose = True
        else:
            m = UCT(rootstate=state, itermax=100, verbose=False)
        print (v)
        print(p)
        brd = list(state.board)
        print(state.board)
        print (brd)
        l=[p, brd , m]
        print(l)
        print("Best Move: " + str(m) + "\n")
        state.DoMove(m)
        print(l)
        currentplay.append(l) # Dataset structure for each player is [ 1. the player(1 or 2), 2. the board states at the moment, 3. the best detected move]
        if state.GetResult(state.playerJustMoved) != False:
            print(str(state))
            break
        print (currentplay)
    if state.GetResult(state.playerJustMoved) == 1.0:
        win = str(state.playerJustMoved)
        print("Player " + str(state.playerJustMoved) + " wins!")
    elif state.GetResult(state.playerJustMoved) == 0.0:
        win = str(3 - state.playerJustMoved)
        print("Player " + str(3 - state.playerJustMoved) + " wins!")
    else: 
        win = 0
        print("Nobody wins!")
    print (win)
    currentplay.append(win)
    print(currentplay) # at the end of each game the winer will be added into the structure.
    data.append(list(currentplay))



In [9]:

data = []
for i in range (0, 10):
    UCTPlayGame()
print(data)
#print(currentplay)


...
...
...



NameError: name 'v' is not defined

Dataset generator


In [12]:
# The actual part

# XO game


import numpy as np


def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    win = 0
    best = []
    currentplay = []
    brd =[]
    l=[]
    count = 0

    state = OXOState() # uncomment to play OXO
    while state.GetMoves() != []:
        #print(str(state))
        #st = state
        p = 3 - state.playerJustMoved
        if state.playerJustMoved == 1:
            m = UCT(rootstate=state, itermax=1000, verbose=False)  # play with values for itermax and verbose = True
        else:
            m = UCT(rootstate=state, itermax=100, verbose=False)
        #print (v)
        #print(p)
        brd = list(state.board)
        #print(state.board)
        #print (brd)
        l=[p, brd , m]
        #print(l)
        #print("Best Move: " + str(m) + "\n")
        state.DoMove(m)
        #print(l)
        currentplay.append(l) # Dataset structure for each player is [ 1. the player(1 or 2), 2. the board states at the moment, 3. the best detected move]
        count += 1
        if state.GetResult(state.playerJustMoved) != False:
            #print(str(state))
            break
        #print (currentplay)
    if count < 9 :
        for i in range (count, 9):
            currentplay.append(np.nan)
    if state.GetResult(state.playerJustMoved) == 1.0:
        win = str(state.playerJustMoved)
        print("Player " + str(state.playerJustMoved) + " wins!")
    elif state.GetResult(state.playerJustMoved) == 0.0:
        win = str(3 - state.playerJustMoved)
        print("Player " + str(3 - state.playerJustMoved) + " wins!")
    else: 
        win = 0
        print("Nobody wins!")
    #print (win)
    currentplay.append(win)
    #print(currentplay) # at the end of each game the winer will be added into the structure.
    data.append(list(currentplay))


In [13]:
data = []
for i in range (0, 150):
    UCTPlayGame()
 #print(data)
 #print(currentplay)

Nobody wins!
Player 1 wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Player 2 wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 2 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 2 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
P

In [14]:
import pandas as pd

gdata = pd.DataFrame(data, columns=['Step 0' , 'Step 1' , 'Step 2' , 'Step 3' , 'Step 4' , 'Step 5' , 'Step 6' , 'Step 7' , 'Step 8' , 'The winner'])

gdata

,Step 0,Step 1,Step 2,Step 3,Step 4,Step 5,Step 6,Step 7,Step 8,The winner
0,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 4]","[2, [0, 0, 0, 0, 1, 0, 0, 0, 0], 0]","[1, [2, 0, 0, 0, 1, 0, 0, 0, 0], 6]","[2, [2, 0, 0, 0, 1, 0, 1, 0, 0], 2]","[1, [2, 0, 2, 0, 1, 0, 1, 0, 0], 1]","[2, [2, 1, 2, 0, 1, 0, 1, 0, 0], 7]","[1, [2, 1, 2, 0, 1, 0, 1, 2, 0], 5]","[2, [2, 1, 2, 0, 1, 1, 1, 2, 0], 3]","[1, [2, 1, 2, 2, 1, 1, 1, 2, 0], 8]",0
1,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 1]","[2, [0, 1, 0, 0, 0, 0, 0, 0, 0], 0]","[1, [2, 1, 0, 0, 0, 0, 0, 0, 0], 4]","[2, [2, 1, 0, 0, 1, 0, 0, 0, 0], 6]","[1, [2, 1, 0, 0, 1, 0, 2, 0, 0], 3]","[2, [2, 1, 0, 1, 1, 0, 2, 0, 0], 7]","[1, [2, 1, 0, 1, 1, 0, 2, 2, 0], 5]",NaN,NaN,1
2,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 6]","[2, [0, 0, 0, 0, 0, 0, 1, 0, 0], 0]","[1, [2, 0, 0, 0, 0, 0, 1, 0, 0], 7]","[2, [2, 0, 0, 0, 0, 0, 1, 1, 0], 4]","[1, [2, 0, 0, 0, 2, 0, 1, 1, 0], 8]",NaN,NaN,NaN,NaN,1
3,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 8]","[2, [0, 0, 0, 0, 0, 0, 0, 0, 1], 0]","[1, [2, 0, 0, 0, 0, 0, 0, 0, 1], 6]","[2, [2, 0, 0, 0, 0, 0, 1, 0, 1], 2]","[1, [2, 0, 2, 0, 0, 0, 1, 0, 1], 1]","[2, [2, 1, 2, 0, 0, 0, 1, 0, 1], 7]","[1, [2, 1, 2, 0, 0, 0, 1, 2, 1], 4]","[2, [2, 1, 2, 0, 1, 0, 1, 2, 1], 3]","[1, [2, 1, 2, 2, 1, 0, 1, 2, 1], 5]",0
4,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 4]","[2, [0, 0, 0, 0, 1, 0, 0, 0, 0], 0]","[1, [2, 0, 0, 0, 1, 0, 0, 0, 0], 6]","[2, [2, 0, 0, 0, 1, 0, 1, 0, 0], 2]","[1, [2, 0, 2, 0, 1, 0, 1, 0, 0], 1]","[2, [2, 1, 2, 0, 1, 0, 1, 0, 0], 7]","[1, [2, 1, 2, 0, 1, 0, 1, 2, 0], 3]","[2, [2, 1, 2, 1, 1, 0, 1, 2, 0], 5]","[1, [2, 1, 2, 1, 1, 2, 1, 2, 0], 8]",0
...,...,...,...,...,...,...,...,...,...,...
145,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 2]","[2, [0, 0, 1, 0, 0, 0, 0, 0, 0], 0]","[1, [2, 0, 1, 0, 0, 0, 0, 0, 0], 4]","[2, [2, 0, 1, 0, 1, 0, 0, 0, 0], 6]","[1, [2, 0, 1, 0, 1, 0, 2, 0, 0], 3]","[2, [2, 0, 1, 1, 1, 0, 2, 0, 0], 5]","[1, [2, 0, 1, 1, 1, 2, 2, 0, 0], 1]","[2, [2, 1, 1, 1, 1, 2, 2, 0, 0], 7]","[1, [2, 1, 1, 1, 1, 2, 2, 2, 0], 8]",0
146,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 0]","[2, [1, 0, 0, 0, 0, 0, 0, 0, 0], 4]","[1, [1, 0, 0, 0, 2, 0, 0, 0, 0], 1]","[2, [1, 1, 0, 0, 2, 0, 0, 0, 0], 2]","[1, [1, 1, 2, 0, 2, 0, 0, 0, 0], 6]","[2, [1, 1, 2, 0, 2, 0, 1, 0, 0], 3]","[1, [1, 1, 2, 2, 2, 0, 1, 0, 0], 5]","[2, [1, 1, 2, 2, 2, 1, 1, 0, 0], 8]","[1, [1, 1, 2, 2, 2, 1, 1, 0, 2], 7]",0
147,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 0]","[2, [1, 0, 0, 0, 0, 0, 0, 0, 0], 4]","[1, [1, 0, 0, 0, 2, 0, 0, 0, 0], 6]","[2, [1, 0, 0, 0, 2, 0, 1, 0, 0], 3]","[1, [1, 0, 0, 2, 2, 0, 1, 0, 0], 1]","[2, [1, 1, 0, 2, 2, 0, 1, 0, 0], 2]","[1, [1, 1, 2, 2, 2, 0, 1, 0, 0], 5]","[2, [1, 1, 2, 2, 2, 1, 1, 0, 0], 8]","[1, [1, 1, 2, 2, 2, 1, 1, 0, 2], 7]",0
148,"[1, [0, 0, 0, 0, 0, 0, 0, 0, 0], 5]","[2, [0, 0, 0, 0, 0, 1, 0, 0, 0], 0]","[1, [2, 0, 0, 0, 0, 1, 0, 0, 0], 4]","[2, [2, 0, 0, 0, 1, 1, 0, 0, 0], 2]","[1, [2, 0, 2, 0, 1, 1, 0, 0, 0], 1]","[2, [2, 1, 2, 0, 1, 1, 0, 0, 0], 3]","[1, [2, 1, 2, 2, 1, 1, 0, 0, 0], 6]","[2, [2, 1, 2, 2, 1, 1, 1, 0, 0], 7]","[1, [2, 1, 2, 2, 1, 1, 1, 2, 0], 8]",0


In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
sh = gc.create('GameDataFrameNo1.csv')

In [0]:
#worksheet = gc.open(r'GameDataFrameNo1.csv')
#worksheet = gdata.to_csv('GameDataFrameNo1.csv',index=False)

In [194]:
import webbrowser

webbrowser.open('https://docs.google.com/spreadsheets/d/1jsrrykZN0buLBcC1QGPTP6abD-sz3xRawdGwuoPRFLI/edit?usp=sharing')



False

In [0]:
gc.open('GameDataFrameNo1.xlsx')

gdata.to_excel('GameDataFrameNo1.xlsx')  
#df.to_excel('M:/Test.xlsx')

In [19]:
gdata.to_csv('M:/GameOutput-1-V1.csv',index=False)

I could not get out puts from google colab into a file nowhere 
So I tried it in the jupyter notebook and got the data set


Trying the new form of the Dataset


In [20]:
# The new dataset part

# XO game


import numpy as np
import pandas as pd



def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    win = 0
    #best = []
    currentplay = []
    brd =[]
    l=[]
    count = 0
    #testing = []
    
    state = OXOState() # uncomment to play OXO
    while state.GetMoves() != []:
        #print(str(state))
        #st = state
        p = 3 - state.playerJustMoved
        if state.playerJustMoved == 1:
            m = UCT(rootstate=state, itermax=1000, verbose=False)  # play with values for itermax and verbose = True
        else:
            m = UCT(rootstate=state, itermax=100, verbose=False)
        #print (v)
        #print(p)
        brd = list(state.board)
        #print(state.board)
        #print (brd)
        #l= [p, brd , m]
        #print(l)
        #print("Best Move: " + str(m) + "\n")
        currentplay.append(int(p))
        currentplay.append(brd)
        currentplay.append(int(m))# append the values
        state.DoMove(m)
        #print(l)
        #currentplay.append(l) # Dataset structure for each player is [ 1. the player(1 or 2), 2. the board states at the moment, 3. the best detected move]
        #testframe = pd.DataFrame(l , columns= None) # , columns=['The player' , 'Current Board' , 'Best Move']
        #testing.append(testframe)
        count += 1
        if state.GetResult(state.playerJustMoved) != False:
            #print(str(state))
            break
        #print (currentplay)
    if count < 9 :# Do this to make data frame nice
        for i in range (count, 9):# a single member is now 3
            currentplay.append(np.nan)
            currentplay.append(np.nan)
            currentplay.append(np.nan)
            #testing.append(np.nan)
    if state.GetResult(state.playerJustMoved) == 1.0:
        win = str(state.playerJustMoved)
        print("Player " + str(state.playerJustMoved) + " wins!")
    elif state.GetResult(state.playerJustMoved) == 0.0:
        win = str(3 - state.playerJustMoved)
        print("Player " + str(3 - state.playerJustMoved) + " wins!")
    else: 
        win = 0
        print("Nobody wins!")
    #print (win)
    currentplay.append(win)
    #testing.append(win)
    #print(currentplay) # at the end of each game the winer will be added into the structure.
    data2.append(list(currentplay))
    #data.append(testing)


In [21]:
 #data = []
data2 = []
for i in range (0, 150):
    UCTPlayGame()
print(data)
 #print(currentplay)

Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 2 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!
Player 1 wins!
Nobody wins!
Nobody wins!
Nobody wins!
Nobody wins!


P = Player ,  brd = Board ,  BM = Best Move , Sx = Stage x of the game

In [22]:
import pandas as pd

gdata2 = pd.DataFrame(data2, columns=['S0 P','S0 Brd','S0 BM', 'S1 P','S1 Brd','S1 BM','S2 P','S2 Brd','S2 BM','S3 P','S3 Brd','S3 BM','S4 P','S4 Brd','S4 BM','S5 P','S5 Brd','S5 BM','S6 P','S6 Brd','S6 BM',
                                    'S7 P','S7 Brd','S7 BM','S8 P','S8 Brd','S8 BM','The winner'])

gdata2

,S0 P,S0 Brd,S0 BM,S1 P,S1 Brd,S1 BM,S2 P,S2 Brd,S2 BM,S3 P,...,S6 P,S6 Brd,S6 BM,S7 P,S7 Brd,S7 BM,S8 P,S8 Brd,S8 BM,The winner
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",4,2,"[0, 0, 0, 0, 1, 0, 0, 0, 0]",0,1,"[2, 0, 0, 0, 1, 0, 0, 0, 0]",1,2,...,1.0,"[2, 1, 0, 1, 1, 2, 0, 2, 0]",2.0,2.0,"[2, 1, 1, 1, 1, 2, 0, 2, 0]",6.0,1.0,"[2, 1, 1, 1, 1, 2, 2, 2, 0]",8.0,0
1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",3,2,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",0,1,"[2, 0, 0, 1, 0, 0, 0, 0, 0]",2,2,...,1.0,"[2, 0, 1, 1, 2, 0, 0, 2, 1]",1.0,2.0,"[2, 1, 1, 1, 2, 0, 0, 2, 1]",5.0,1.0,"[2, 1, 1, 1, 2, 2, 0, 2, 1]",6.0,0
2,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",4,2,"[0, 0, 0, 0, 1, 0, 0, 0, 0]",0,1,"[2, 0, 0, 0, 1, 0, 0, 0, 0]",2,2,...,1.0,"[2, 0, 1, 1, 1, 2, 2, 0, 0]",1.0,2.0,"[2, 1, 1, 1, 1, 2, 2, 0, 0]",7.0,1.0,"[2, 1, 1, 1, 1, 2, 2, 2, 0]",8.0,0
3,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,2,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",4,1,"[1, 0, 0, 0, 2, 0, 0, 0, 0]",6,2,...,1.0,"[1, 0, 0, 2, 2, 1, 1, 2, 0]",1.0,2.0,"[1, 1, 0, 2, 2, 1, 1, 2, 0]",2.0,1.0,"[1, 1, 2, 2, 2, 1, 1, 2, 0]",8.0,0
4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,2,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",4,1,"[1, 0, 0, 0, 2, 0, 0, 0, 0]",1,2,...,1.0,"[1, 1, 2, 2, 2, 0, 1, 0, 0]",5.0,2.0,"[1, 1, 2, 2, 2, 1, 1, 0, 0]",8.0,1.0,"[1, 1, 2, 2, 2, 1, 1, 0, 2]",7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,2,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",4,1,"[1, 0, 0, 0, 2, 0, 0, 0, 0]",1,2,...,1.0,"[1, 1, 2, 2, 2, 0, 1, 0, 0]",5.0,2.0,"[1, 1, 2, 2, 2, 1, 1, 0, 0]",8.0,1.0,"[1, 1, 2, 2, 2, 1, 1, 0, 2]",7.0,0
146,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",2,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0]",0,1,"[2, 0, 1, 0, 0, 0, 0, 0, 0]",4,2,...,1.0,"[2, 0, 1, 1, 1, 2, 2, 0, 0]",7.0,2.0,"[2, 0, 1, 1, 1, 2, 2, 1, 0]",1.0,1.0,"[2, 2, 1, 1, 1, 2, 2, 1, 0]",8.0,0
147,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",2,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0]",0,1,"[2, 0, 1, 0, 0, 0, 0, 0, 0]",4,2,...,1.0,"[2, 0, 1, 1, 1, 2, 2, 0, 0]",1.0,2.0,"[2, 1, 1, 1, 1, 2, 2, 0, 0]",7.0,1.0,"[2, 1, 1, 1, 1, 2, 2, 2, 0]",8.0,0
148,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,2,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",4,1,"[1, 0, 0, 0, 2, 0, 0, 0, 0]",3,2,...,1.0,"[1, 2, 1, 1, 2, 0, 2, 0, 0]",7.0,2.0,"[1, 2, 1, 1, 2, 0, 2, 1, 0]",8.0,1.0,"[1, 2, 1, 1, 2, 0, 2, 1, 2]",5.0,0


In [24]:
#df.to_excel('M:/Test.xlsx')

gdata2.to_csv('M:/GameOutput-2-V1.csv',index=False)